In [ ]:
data_dir = ".\\data\\"


# SLC-1730, air, slow, 10mins
#filename = "2018-07-25-10-58-32.tdms"
#keyence_filename= "2018-07-25-10-58-24-KEYENCE.tdms"

# SLC-1730, vac, 24hrs
filename = "2018-07-26-14-39-24.tdms"
keyence_filename= None

In [ ]:
import pickle

if keyence_filename is not None:
    pickle_file = data_dir + "_".join([filename, keyence_filename, "proc_data.p"])
else:
    pickle_file = data_dir + "_".join([filename, "proc_data.p"])

try:
    full_data = pickle.load(open(pickle_file, 'rb'))
    print("Loaded the data from " + pickle_file)
except FileNotFoundError:
    print("File not found, load the data again!")

In [ ]:
# Pop the info out, it can't be processed...
info = full_data.pop("info")

# Print the info
col_width = max(len(k) for k in info.keys()) + 2  # padding
for k in info:
    print("".join([k.ljust(col_width), info[k]]))

In [ ]:
import data_tools
from pprint import pprint

truncate = False

if truncate:

    print(len(full_data['set_point']['time']))

    limits = [full_data['set_point']['time'][0], full_data['set_point']['time'][1000]]

    print("Using limits:")
    pprint(limits)

    # Truncate the data
    data = dict(zip(data.keys(), [data_tools.truncate(limits, full_data[k]) for k in full_data]))

else:
    data = full_data

print(len(data['set_point']['time']))

In [ ]:
# Loop through the SPs. Average the values between two set points.

import numpy as np

from importlib import reload
reload(data_tools)

k_agg = ['position', 'data_set_point_data']
k_win = ['set_point', 'time']

# Make a progress bar
from ipywidgets import FloatProgress
from IPython.display import display
print("Calculating averages")
p = FloatProgress()
display(p)

from time import time

start = time()

data_tools.aggregate_data(k_agg, k_win, data, p)

duration = time() - start
        
print("Done in {}s".format(duration))
    

In [ ]:
for k in data:
    print(k)
    for l in data[k]:
        print("\t{}\t{}".format(l, len(data[k][l])))

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=True)

plotter = Scatter
#plotter = Scattergl

plots = [
    plotter(x=data['position']['time'], y=np.ma.MaskedArray(data['position']['data_set_point_data'], 
                                                            mask=data['position']['mask_set_point']), name="Stage Error"),
    plotter(x=data['set_point']['time'], y=data['set_point']['avg_position_data_set_point_data'], name="Stage Error Average"),
    plotter(x=data['set_point']['time'], y=data['set_point']['max_position_data_set_point_data'], name="Stage Error Max"),
    plotter(x=data['set_point']['time'], y=data['set_point']['min_position_data_set_point_data'], name="Stage Error Min"),

]

layout = Layout(
    width = 800,
    height = 800,
    title = filename,
    xaxis = dict(
        title = "Time",
    ),
    yaxis = dict(
        title = "Position (mm)",
    )
)

fig = Figure(data=plots, layout=layout)
iplot(fig, filename=filename)